In [77]:
from imports import *

# Scraping


! python scrape_article_urls.py

! python scrape_article_data.py

# Processing

In [78]:
df=pd.read_csv('data/daily_df.csv')

In [79]:
df=df.drop_duplicates(subset='Article ID')

In [81]:
#article ID
df['Article ID']=df['Article ID'].str.replace('ID|-','',regex=True).str.strip()

In [82]:
# Date Published and Time Scraped
month_dict={
 'იან':'January',
 'თებ':'February',
 'მარ':'March',
 'აპრ':'April',
 'მაი':'May',
 'ივნ':'June',
 'ივლ':'July',
 'აგვ':'August',
 'სექ':'September',
 'ოქტ':'October',
 'ნოე':'November',
 'დეკ':'December'
}

df['Date Published']=df['Date Published'].replace(month_dict, regex=True)
df['Date Published']=pd.to_datetime(df['Date Published'],errors='coerce')

time_scraped=df['Time Scraped'].tolist()
df=df.drop(columns='Time Scraped')
df.insert(
    df.columns.get_loc('Date Published')+1,
    'Time Scraped',
    time_scraped
)

In [83]:
#Addresses
df.drop('General Address',axis=1,errors='ignore',inplace=True)
df.insert(
    df.columns.get_loc('Address'),
    'General Address',
    df['Title'].apply(lambda x:x.split('ბინა ')[-1] if isinstance(x,str) else None)
)
suburb_dict={
    'აეროპორტ': 'აეროპორტის დასახლება',
    'ავჭალ': 'ავაჭალა',
    'ავლაბარ': 'ავლაბარი',
    'აფრიკ': 'აფრიკის დასახლება',
    'ბაგებ': 'ბაგები',
    'გლდან': 'გლდანი',
    'დიდ დიღომ': 'დიდი დიღომი',
    'დიდი დიღომ': 'დიდი დიღომი',
    'დიდუბ': 'დიდუბე',
    'დიღმის მასივ': 'დიღმის მასივი',
    'მასივ': 'დიღმის მასივი',
    'დიღომი 1': 'დიღომი',
    'ვაზისუბ': 'ვაზისუბანი',
    'ვაკე': 'ვაკე',
    'ვაჟა': 'ვაჟა-ფშაველა',
    'ფშაველა': 'ვაჟა-ფშაველა',
    'ვარკეთილ': 'ვარკეთილი',
    'ვაშლიჯ': 'ვაშლიჯვარი',
    'ვერა': 'ვერა',
    'ვეძის': 'ვეძისი',
    'ზაჰეს': 'ზაჰესი',
    'თემქ': 'თემქა',
    'ივერთუბ': 'ივერთუბანი',
    'ისან': 'ისანი',
    'ლილო': 'ლილოს დასახლება',
    'ლისის': 'ლისის ტბის დასახლება',
    'ლისზე': 'ლისის ტბის დასახლება',
    'მთაწმინდ': 'მთაწმინდა',
    'მუხიან': 'მუხიანი',
    'ნავთლუღ': 'ნავთლუღი',
    'ნაძალად': 'ნაძალადევი',
    'ნახალოვ': 'ნახალოვკა',
    'ნუცუბიძ': 'ნუცუბიძის პლატო',
    'ორთაჭალ': 'ორთაჭალა',
    'ორხევ': 'ორხევი',
    'საბურთალ': 'საბურთალო',
    'სამგორ': 'სამგორი',
    'სანზონ': 'სანზონა',
    'სოლოლაკ': 'სოლოლაკი',
    'სოფელ დიღომ': 'სოფელი დიღომი',
    'სოფელი დიღომ': 'სოფელი დიღომი',
    'სოფ. დიღო': 'სოფელი დიღომი',
    'ფონიჭ': 'ფონიჭალა',
    'ჩუღურეთ': 'ჩუღურეთი',
    'ძველ თბილი': 'ძველი თბილისი',
    'ძველი თბილი': 'ძველი თბილისი'
 }

def contains_key(x):
    for key, value in suburb_dict.items():
        if key in str(x):
            return value
    return None

In [84]:
#make more general
df['General Address'] = df['General Address'].apply(contains_key)

df.drop('Detailed Address',axis=1,errors='ignore',inplace=True)
df.insert(
    df.columns.get_loc('Address'),
    'Detailed Address',
    df['Address'].apply(lambda x:x.split(', ')[-1] if isinstance(x,str) else None)
)


In [85]:
# Details_1 and Dedails_2
df['Area SQMT']=df['Details_1'].astype(str).apply(lambda x:x.split('საერთო ფართი|')[-1].split(' ')[0] if isinstance(x,str) else None).apply(pd.to_numeric,errors='coerce')
df['Bedrooms']=df['Details_1'].astype(str).apply(lambda x:x.split('საძინებელი|')[-1].split("'")[0] if isinstance(x,str) else None).apply(pd.to_numeric,errors='coerce')
df['Rooms']=df['Details_1'].astype(str).apply(lambda x:x.split('ოთახი|')[-1].split("'")[0] if isinstance(x,str) else None).apply(pd.to_numeric,errors='coerce')
df['Floor']=df['Details_1'].astype(str).apply(lambda x:x.split('სართული|')[-1].split("/")[0] if isinstance(x,str) else None).apply(pd.to_numeric,errors='coerce')
df['Max Floors']=df['Details_1'].astype(str).apply(lambda x:x.split('სართული|')[-1].split("/")[-1].split("'")[0] if isinstance(x,str) else None).apply(pd.to_numeric,errors='coerce')
df['Condition']=df['Details_2'].astype(str).apply(lambda x:x.split('მდგომარეობა|')[-1].split("/")[-1].split("'")[0] if isinstance(x,str) else None)
df['Project']=df['Details_2'].astype(str).apply(lambda x:x.split('პროექტი|')[-1].split("/")[-1].split("'")[0] if isinstance(x,str) else None)
df['Bathrooms']=df['Details_2'].astype(str).apply(lambda x:x.split('სველი წერტილი|')[-1].split("'")[0] if isinstance(x,str) else None).apply(pd.to_numeric,errors='coerce')
df['Status']=df['Details_2'].astype(str).apply(lambda x:x.split('სტატუსი|')[-1].split("/")[-1].split("'")[0] if isinstance(x,str) else None)

df=df.drop(columns=['Details_1','Details_2'])

In [86]:
#Price
df['Price USD']=df.apply(lambda x:
                  float(x['Price'].split('$')[0].replace(',','')) if '$' in str(x) else 
                  float(x['Price'].split('₾')[0].replace(',',''))/x['USD-GEL Rate'] if '₾' in str(x) else None, axis=1
                  )
df=df.drop(columns='Price')

df=df[(df['Price USD']>0)&(df['Area SQMT']>0)]
df['Price Per SQMT']=(df['Price USD']/df['Area SQMT']).round(2)
df=df.dropna(subset=['Price Per SQMT'])

mean_price=df['Price Per SQMT'].mean()
std_price=df['Price Per SQMT'].mean()
df=df.query('@mean_price-5*@std_price<`Price Per SQMT`<@mean_price+5*@std_price')

In [87]:
#filter out unwanted rows
df=df[(df['Condition']!='{')&(df['Status']!='{')]

In [88]:
#save the processed df

daily_df_processed=df
daily_df_processed.to_csv('data/daily_df_processed.csv',index=False)
print('Finished preprocessing')

Finished preprocessing


# Groupby

In [90]:
df=pd.read_csv('data/daily_df_processed.csv')

df['Date Published']=pd.to_datetime(df['Date Published'],errors='coerce')

grouped_df=df.groupby([df['Date Published'].dt.date,'General Address','Status','Condition']).agg({'Article ID':'count','Price Per SQMT':'median'}).reset_index()

all_time_grouped=pd.read_csv(os.path.join('dash','all_time_grouped.csv'))
all_time_grouped=all_time_grouped.drop_duplicates(subset=['Date Published','General Address','Status','Condition'],keep='last')
all_time_grouped=pd.concat([all_time_grouped,grouped_df],ignore_index=True)
all_time_grouped=all_time_grouped.sort_values(['Date Published','General Address','Status','Condition','Article ID'])
all_time_grouped=all_time_grouped.drop_duplicates(subset=['Date Published','General Address','Status','Condition'],keep='last')
all_time_grouped=all_time_grouped.groupby(['Date Published','Status']).filter(lambda x:x['Article ID'].sum()>=10)
all_time_grouped['Date Published']=pd.to_datetime(all_time_grouped['Date Published'],errors='coerce')
all_time_grouped=all_time_grouped.sort_values('Date Published')
all_time_grouped.to_csv(os.path.join('dash','all_time_grouped.csv'),index=False)
all_time_grouped.to_csv(os.path.join('data','all_time_grouped.csv'),index=False)


print('Finished all_time_grouped.csv')

Finished all_time_grouped.csv


# Coordinates

In [91]:
df=pd.read_csv('data/all_time_unprocessed.csv')[['Address','Date Published','Price','Details_1']]

In [92]:
df['address_simplified']=df['Address'].apply(lambda x:
                    'თბილისი,'+x.split('ქ.')[0].split('გამზ')[0].replace(' 0','').replace(' 1','').replace(' 2','').replace(' 3','').split(',')[-1] if isinstance(x,str) else None
                    )

In [93]:
# date
month_dict={
 'იან':'January',
 'თებ':'February',
 'მარ':'March',
 'აპრ':'April',
 'მაი':'May',
 'ივნ':'June',
 'ივლ':'July',
 'აგვ':'August',
 'სექ':'September',
 'ოქტ':'October',
 'ნოე':'November',
 'დეკ':'December'
}

df['Date Published']=df['Date Published'].replace(month_dict, regex=True)
df['Date Published']=pd.to_datetime(df['Date Published'],errors='coerce')

#area
df['Area SQMT']=df['Details_1'].astype(str).apply(lambda x:x.split('საერთო ფართი|')[-1].split(' ')[0] if isinstance(x,str) else None).apply(pd.to_numeric,errors='coerce')

#price
df['Price USD']=df.apply(lambda x:
                  float(x['Price'].split('$')[0].replace(',','')) if '$' in str(x) else None, axis=1
)
df=df.drop(columns='Price')

df=df[(df['Price USD']>0)&(df['Area SQMT']>0)]
df['Price Per SQMT']=(df['Price USD']/df['Area SQMT']).round(2)
df=df.dropna(subset=['Price Per SQMT'])

mean_price=df['Price Per SQMT'].mean()
std_price=df['Price Per SQMT'].mean()
df=df.query('@mean_price-5*@std_price<`Price Per SQMT`<@mean_price+5*@std_price')

In [94]:
coordinates_dict=pickle.load(open("data/coordinates_dict.pickle",'rb'))

In [95]:
def map_coordinates(address):
    if address in coordinates_dict:
        return coordinates_dict[address]
    else:
        return {'Lat': None, 'Lon': None, 'Street': None}
    
mapped_coordinates=df['address_simplified'].apply(map_coordinates)
mapped_df=pd.DataFrame(mapped_coordinates.tolist(),index=df.index)
df=pd.concat([df,mapped_df],axis=1)

In [96]:
coordinates_df=df[['Date Published','Price Per SQMT','Lat','Lon','Street']]

In [97]:
coordinates_df.to_csv('data/coordinates_df.csv',index=False)